## **DATA EXTRACTION**
### Source: [**UN COMTRADE DATABASE**](https://https://comtrade.un.org/)
The purpose behind the following code is to be able to extract all data regarding Mexico's international operations (imports & exports).

After exploring the data through different queries in Postman, the following insights were gathered:

* Mexico started reporting its operations until 1990, so prior to this year, all data is unavailable. 

*   From 1990 to 2009, all data reported by Mexico was recorded by Year, so we are not going to be able to explore how trade perfomed month by month in this time period. 

Keep in mind the following ...

*   Public Users have only access to 1000 queries per hour, 1 query per second.
*Queries should be specific and simple. 

So, the challenge was to create a program that automatically extracts the data considering all the restrictions that we Public Users have.

In [1]:
import os
import requests
import json
import pandas as pd
import time 
import pymysql
from sqlalchemy import create_engine
import pyautogui # pip install pyautogui

In [2]:
def query(dicc):
  
  """Returns a string formatted as the query that it is needed for the API-CALL"""

  # Input: dictionary with the specifications to extract data
  # --> Output: Query (string) to use.
  
  dicc = json.dumps(dicc)
  remove = '{}" '

  for r in remove:
    dicc = dicc.replace(r, '').replace(':', '=').replace(',','&')
    query = dicc

  return query

In [3]:
def months(year=int, period=None):

  """
  Returns a list of strings that represent each month of a year.
  Since we can't use complex queries, we have to be specific in the time
  period from which we are attempting to get data. For this reason, 
  this function was created for the purpose of iterating it to create
  a list of queries (each query having a different time period specified).
  """

  # Input: Year = 1990
  # --> Output: List of Time Periods = [199001, 199002, 199003, 199004 ... 199012]

  keys = ['JANUARY', 'FEBRUARY', 'MARCH', 'APRIL', 'MAY', 'JUNE', 'JULY',\
          'AUGUST', 'SEPTEMBER', 'OCTOBER',  'NOVEMBER', 'DECEMBER']
  values = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
  months = {keys[i]:values[i] for i in range(len(keys))}

  if period==None:
    return [str(year)+month for month in months.values()]

  if type(period) == str:
    return str(year)+months[period]

In [3]:
def dicc_zones():
    
    """
    Returns a dictionary with all the ids and names for each of the zones/areas
    that report data to the UN COMTRADE.
    """

    # --> Output: Dictionary = {'id':'zone'} = {'484':'Mexico'}

    zones = requests.get('https://comtrade.un.org/Data/cache/partnerAreas.json')
    zones = zones.json()['results'] 
    
    values = []
    
    for zone in zones:
      for (k,v) in zone.items():
        values.append(v)
    
    values = iter(values)
    id_zone_dicc = dict(zip(values, values))

    return id_zone_dicc

In [4]:
def all_query(time=list, zone='484'):
  
  """
  Returns a list of generated queries.
  Right now the function is formatted strictly to get all the totals of all 
  the commodities that were traded by Mexico since 1990, which is the year
  that Mexico started reporting its data to the United Nations.
  """

  # Input: years to be explored 
  # --> Output: List of Time Periods = [199001, 199002, 199003, 199004 ... 199012]

  period_years = list(range(1990, 2010))
  period_months = list(range(2010, 2021))

  check_years = all(item in period_years for item in time)
  check_months =  all(item in period_months for item in time)

  if check_months == True:
    freq = 'M'
  if check_years == True:
    freq = 'A'

  list_of_queries = []

  for year in time:
    
    keys = ['type', 'freq', 'px', 'ps', 'r', 'p', 'rg', 'cc']
    values = ['C', freq, 'HS', str(year), zone, 'all', 'all', 'TOTAL']
    
    query_dict = {keys[i]:values[i] for i in range(len(keys))}
    query_final = query(query_dict)
    
    list_of_queries.append(query_final)

  return list_of_queries

In [5]:
def api_getting(api=str, queries=list):
  
  """
  Returns a concatenated dataframe with all data collected
  (from each query a dataframe is created)
  """

  # Input: api, list of specific queries
  # --> Output: Concatenated Pandas Dataframe with all the data collected.

  dfs = []

  for query in queries:

    response = requests.get(f'{api}{query}')
    time.sleep(1)
    data = pd.DataFrame(response.json()['dataset'])
    dfs.append(data)
  
  dataframe_api = pd.concat(dfs, axis=0, ignore_index=True)  

  return dataframe_api

In [6]:
def data_extraction():

  """
  Returns the results in a concatenated dataframe.
  Right now the function is formatted to gather all data reported by Mexico
  from 1990 to 2020. Remember that the data from 1990 to 2009 was reported 
  annually, and from 2010 to 2020, we have access to the data of each month.
  """

  """
  The difference between the past function and this one relies on the fact
  that this one is arranged to extract all data reported by Mexico. 
  The past function is open to receive any type of queries regarding any other
  country or a specific time period.
  """

  time_months = list(range(2010, 2021))
  time_years = list(range(1990, 2010))

  final_queries_months = all_query(time_months)
  final_queries_years = all_query(time_years)

  api = 'http://comtrade.un.org/api/get?'
  
  results_months = api_getting(api, final_queries_months)
  results_years = api_getting(api, final_queries_years)

  results = pd.concat([results_months, results_years], axis=0, ignore_index=True)

  return results

## **DATA CLEANING** 

The following code is to clean and prepare the data for analysis. 

(Null & NaN values were deleted, and all data was restructured to be a more user-friendly dataframe)

Every query that we used by running the past code, was converted into a Pandas Object.

In [7]:
def data_prettifier(dataframe):

  """
  Returns a dataframe cleaned, by eliminating unnecessary/repeated data, and 
  also by formatting data so that we can later perform various analyses through
  pivot tables and graphs ...
  """

  # Input: Dataframe with all the raw data extracted from the API.
  # --> Output: Concatenated Pandas Dataframe with useful data, in a reliable format.

  data_copy = dataframe.copy()

  data_copy = data_copy[['period', 'rgDesc', 'rtTitle', 'ptTitle', 'cmdDescE', 'TradeValue']]
    
  data_copy.columns = ['Period', 'Regimen', 'Country', 'Partner','Commodity', 'Trade Value']
    
  data_copy['Period'] = data_copy['Period'].astype(str)
  
  months = data_copy[data_copy['Period'].apply(lambda x: len(x)==6)].index
  years = data_copy[data_copy['Period'].apply(lambda x: len(x)==4)].index

  data_copy.loc[months,'Period'] = pd.to_datetime(data_copy.loc[months,'Period'], format='%Y%m', errors='coerce').dt.to_period('m')
  data_copy.loc[years,'Period'] = pd.to_datetime(data_copy.loc[years,'Period'], format='%Y', errors='coerce').dt.to_period('Y')

  data_copy['Trade Value'] = data_copy['Trade Value'].astype(float)

  return data_copy

In [8]:
def data_tosql(pretty_dataframe, engine=None, dbname=None):
    
    if engine == None and dbname == None:
        
        hostname = pyautogui.password(text='HOSTNAME', title='CREDENTIALS', mask='*')
        user = pyautogui.prompt(text='SQL User:', title='USER' , default='Enter here your username ...')
        password = pyautogui.password(text='SQL User Password', title='PASSWORD', mask='*')

        conn = pymysql.connect(
            host=hostname,
            user=user,
            password=password)

        dbname = input('Name for New Database: ')

        conn.cursor().execute(f'CREATE DATABASE {dbname}')

        conn.close()
        
        credentials = f'@{hostname}/{dbname}'
        
        engine = create_engine(f'mysql+pymysql://{user}:{str(password)}{credentials}')
        
    return pretty_dataframe.to_sql(input('Name of Table: '), engine)

In [10]:
trade_data = data_extraction()
trade_data

,pfCode,yr,period,periodDesc,aggrLevel,IsLeaf,rgCode,rgDesc,rtCode,rtTitle,...,qtAltCode,qtAltDesc,TradeQuantity,AltQuantity,NetWeight,GrossWeight,TradeValue,CIFValue,FOBValue,estCode
0,HS,2010,201008,August 2010,0,0,1,Imports,484,Mexico,...,None,,None,None,None,None,5887887,None,None,0
1,HS,2010,201008,August 2010,0,0,2,Exports,484,Mexico,...,None,,None,None,None,None,33708650,None,None,0
2,HS,2010,201008,August 2010,0,0,1,Imports,484,Mexico,...,None,,None,None,None,None,82538966,None,None,0
3,HS,2010,201003,March 2010,0,0,1,Imports,484,Mexico,...,None,,None,None,None,None,31778856,None,None,0
4,HS,2010,201003,March 2010,0,0,2,Exports,484,Mexico,...,None,,None,None,None,None,2216020,None,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43645,H3,2009,2009,2009,0,0,2,Export,484,Mexico,...,None,,None,None,None,None,666470,None,None,0
43646,H3,2009,2009,2009,0,0,1,Import,484,Mexico,...,None,,None,None,None,None,53846,None,None,0
43647,H3,2009,2009,2009,0,0,2,Export,484,Mexico,...,None,,None,None,None,None,1032399,None,None,0
43648,H3,2009,2009,2009,0,0,1,Import,484,Mexico,...,None,,None,None,None,None,50444097,None,None,0


In [11]:
pretty_trade_data = data_prettifier(trade_data)
pretty_trade_data

,Period,Regimen,Country,Partner,Commodity,Trade Value
0,2010-08,Imports,Mexico,Cambodia,All Commodities,5887887.0
1,2010-08,Exports,Mexico,"China, Hong Kong SAR",All Commodities,33708650.0
2,2010-08,Imports,Mexico,South Africa,All Commodities,82538966.0
3,2010-03,Imports,Mexico,Hungary,All Commodities,31778856.0
4,2010-03,Exports,Mexico,Portugal,All Commodities,2216020.0
...,...,...,...,...,...,...
43645,2009,Export,Mexico,Yemen,All Commodities,666470.0
43646,2009,Import,Mexico,Zambia,All Commodities,53846.0
43647,2009,Export,Mexico,Zambia,All Commodities,1032399.0
43648,2009,Import,Mexico,"Areas, nes",All Commodities,50444097.0


In [18]:
pretty_trade_data.to_json(f'{os.getcwd()}\\mexico-trade-data.json', orient='records')